# Translate isoforms to get protein products

In [18]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

import itertools
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Alphabet import generic_dna, generic_protein
from collections import defaultdict

from outrigger.region import Region

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper/'.format(folder)


splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(csv_folder), index_col=0)

In [10]:
db_filename = '{}/index/gtf/gencode.v19.annotation.gtf.db'.format(outrigger_folder)

db = gffutils.FeatureDB(db_filename)

In [4]:
# splicing_feature_data.head()

## Translate isoform

In [5]:
prefix = 'isoform_translations'
translated_fasta = '{}/{}.fa'.format(alternative_feature_folder, prefix)

In [6]:
outrigger_folder = '{}/outrigger_v2'.format(folder)
! tree $outrigger_folder

/projects/ps-yeolab/obotvinnik/singlecell_pnms/outrigger_v2
|-- index
|   |-- gtf
|   |   |-- gencode.v19.annotation.gtf.db
|   |   `-- novel_exons.gtf
|   |-- junction_exon_direction_triples.csv
|   |-- mxe
|   |   |-- event.bed
|   |   |-- events.csv
|   |   |-- exon1.bed
|   |   |-- exon2.bed
|   |   |-- exon3.bed
|   |   |-- exon4.bed
|   |   |-- intron.bed
|   |   |-- manual_metadata.csv
|   |   |-- splice_sites.csv
|   |   `-- validated
|   |       `-- events.csv
|   `-- se
|       |-- event.bed
|       |-- events.csv
|       |-- exon1.bed
|       |-- exon2.bed
|       |-- exon3.bed
|       |-- intron.bed
|       |-- manual_metadata.csv
|       |-- splice_sites.csv
|       `-- validated
|           |-- events.csv
|           `-- events.csv.bak
|-- junctions
|   |-- metadata.csv
|   `-- reads.csv
`-- psi
    |-- mxe
    |   |-- psi.csv
    |   `-- summary.csv
    |-- outrigger_psi.csv
    |-- se
    |   |-- psi.csv
    |   `-- summary.csv
    `-- summary_formatted.csv

10 director

In [7]:
from outrigger.common import SPLICE_TYPE_ISOFORM_EXONS

In [12]:
pd.options.display.max_columns = 50

In [13]:
se_metadata = pd.read_csv('{}/index/se/events.csv'.format(outrigger_folder), index_col=0)
print(se_metadata.shape)
se_metadata.head()

(403768, 45)


,event_length,event_location,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,exons,illegal_junctions,intron_length,intron_location,isoform1_ccdsid,isoform1_gene_id,isoform1_gene_name,isoform1_gene_status,isoform1_gene_type,isoform1_havana_gene,isoform1_havana_transcript,isoform1_level,isoform1_ont,isoform1_tag,isoform1_transcript_id,isoform1_transcript_name,isoform1_transcript_status,isoform1_transcript_type,isoform2_ccdsid,isoform2_gene_id,isoform2_gene_name,isoform2_gene_status,isoform2_gene_type,isoform2_havana_gene,isoform2_havana_transcript,isoform2_level,isoform2_ont,isoform2_tag,isoform2_transcript_id,isoform2_transcript_name,isoform2_transcript_status,isoform2_transcript_type,junction12,junction13,junction23,junctions,strand
event_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
isoform1=junction:chr12:53689424-53690237:+|isoform2=junction:chr12:53689424-53689622:+@exon:chr12:53689623-53689725:+@junction:chr12:53689726-53690237:+,970.0,event:chr12:53689366-53690335:+,exon:chr12:53689366-53689423:+,58.0,exon:chr12:53689623-53689725:+,103.0,exon:chr12:53690238-53690335:+,98.0,exon:chr12:53689366-53689423:+@exon:chr12:5368...,NaN,816.0,intron:chr12:53689423-53690238:+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,junction:chr12:53689424-53689622:+,junction:chr12:53689424-53690237:+,junction:chr12:53689726-53690237:+,junction:chr12:53689424-53689622:+@junction:ch...,+
isoform1=junction:chr1:32669647-32670198:+|isoform2=junction:chr1:32669647-32669786:+@exon:chr1:32669787-32669980:+@junction:chr1:32669981-32670198:+,1509.0,event:chr1:32669480-32670988:+,exon:chr1:32669480-32669646:+,167.0,exon:chr1:32669787-32669980:+,194.0,exon:chr1:32670199-32670988:+,790.0,exon:chr1:32669480-32669646:+@exon:chr1:326697...,NaN,554.0,intron:chr1:32669646-32670199:+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,junction:chr1:32669647-32669786:+,junction:chr1:32669647-32670198:+,junction:chr1:32669981-32670198:+,junction:chr1:32669647-32669786:+@junction:chr...,+
isoform1=junction:chrX:102931980-102940098:-|isoform2=junction:chrX:102933549-102940098:-@exon:chrX:102933427-102933548:-@junction:chrX:102931980-102933426:-,9727.0,event:chrX:102931423-102941149:-,exon:chrX:102940099-102941149:-,1051.0,exon:chrX:102933427-102933548:-,122.0,exon:chrX:102931423-102931979:-,557.0,exon:chrX:102940099-102941149:-@exon:chrX:1029...,NaN,8121.0,intron:chrX:102931979-102940099:-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,junction:chrX:102933549-102940098:-,junction:chrX:102931980-102940098:-,junction:chrX:102931980-102933426:-,junction:chrX:102933549-102940098:-@junction:c...,-
isoform1=junction:chr11:118968008-118968564:-|isoform2=junction:chr11:118968262-118968564:-@exon:chr11:118968174-118968261:-@junction:chr11:118968008-118968173:-,1409.0,event:chr11:118967345-118968753:-,exon:chr11:118968565-118968753:-,189.0,exon:chr11:118968174-118968261:-,88.0,exon:chr11:118967345-118968007:-,663.0,exon:chr11:118968565-118968753:-@exon:chr11:11...,NaN,559.0,intron:chr11:118968007-118968565:-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,junction:chr11:118968262-118968564:-,junction:chr11:118968008-118968564:-,junction:chr11:118968008-118968173:-,junction:chr11:118968262-118968564:-@junction:...,-
isoform1=junction:chr10:99213421-99214470:+|isoform2=junction:chr10:99213421-99213555:+@exon:chr10:99213556-99213603:+@junction:chr10:99213604-99214470:+,1246.0,event:chr10:99213311-99214556:+,exon:chr10:99213311-99213420:+,110.0,exon:chr10:99213556-99213603:+,48.0,exon:chr10:99214471-99214556:+,86.0,exon:chr10:99213311-99213420:+@exon:chr10:9921...,NaN,1052.0,intron:chr10:99213420-99214471:+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [15]:
len(se_metadata.groupby(level=0, axis=0).size())

32545

In [11]:
list(db.region('chr12:53689366-53689423', featuretype='gene'))

[<Feature gene (chr12:53689075-53693234[+]) at 0x2b586c0e9690>]

In [27]:
list(db.region('chr12:53689366-53689423', featuretype='CDS', completely_within=True))

[<Feature CDS (chr12:53689366-53689423[+]) at 0x2b587e182190>]

In [16]:
list(db.region('chr12:53689366-53689423', featuretype='transcript'))

[<Feature transcript (chr12:53689075-53693234[+]) at 0x2b586c0ea3d0>,
 <Feature transcript (chr12:53689309-53693229[+]) at 0x2b586c0e9710>,
 <Feature transcript (chr12:53689309-53693234[+]) at 0x2b5893eee110>,
 <Feature transcript (chr12:53689311-53690108[+]) at 0x2b5893eee150>,
 <Feature transcript (chr12:53689320-53693234[+]) at 0x2b5893eeee50>,
 <Feature transcript (chr12:53689326-53690490[+]) at 0x2b5893eee4d0>,
 <Feature transcript (chr12:53689326-53693230[+]) at 0x2b5893eee450>,
 <Feature transcript (chr12:53689326-53693234[+]) at 0x2b5893eeedd0>,
 <Feature transcript (chr12:53689326-53693234[+]) at 0x2b5893eeebd0>,
 <Feature transcript (chr12:53689332-53691549[+]) at 0x2b5893eeeb10>,
 <Feature transcript (chr12:53689336-53693234[+]) at 0x2b5893eeea10>,
 <Feature transcript (chr12:53689340-53693234[+]) at 0x2b5893eee910>,
 <Feature transcript (chr12:53689343-53691708[+]) at 0x2b5893eee750>,
 <Feature transcript (chr12:53689360-53691188[+]) at 0x2b5893eee690>,
 <Feature transcript

In [8]:
# mxe_metadata = pd.read_csv('{}/index/mxe/manual_metadata.csv'.format(outrigger_folder), index_col=0)
# print(mxe_metadata.shape)
# mxe_metadata.head()

In [9]:
se_metadata.head(20).groupby(level=0, axis=0).size()

isoform1=junction:chr10:100189400-100190327:-|isoform2=junction:chr10:100189647-100190327:-@exon:chr10:100189548-100189646:-@junction:chr10:100189400-100189547:-          9
isoform1=junction:chr10:100190428-100193696:-|isoform2=junction:chr10:100191049-100193696:-@exon:chr10:100190888-100191048:-@junction:chr10:100190428-100190887:-          1
isoform1=junction:chr10:100193849-100195391:-|isoform2=junction:chr10:100195172-100195391:-@exon:chr10:100195029-100195171:-@junction:chr10:100193849-100195028:-          4
isoform1=junction:chr10:100193849-100195391:-|isoform2=junction:chr10:100195172-100195391:-@novel_exon:chr10:100195101-100195171:-@junction:chr10:100193849-100195100:-    4
isoform1=junction:chr10:101163392-101165512:-|isoform2=junction:chr10:101163632-101165512:-@exon:chr10:101163481-101163631:-@junction:chr10:101163392-101163480:-          1
isoform1=junction:chr10:101191023-101193302:+|isoform2=junction:chr10:101191023-101191686:+@novel_exon:chr10:101191687-101191705:+@junc

In [11]:
SPLICE_TYPE_ISOFORM_EXONS

{'mxe': {'isoform1': ['exon1', 'exon3', 'exon4'],
  'isoform2': ['exon1', 'exon2', 'exon4']},
 'se': {'isoform1': ['exon1', 'exon3'],
  'isoform2': ['exon1', 'exon2', 'exon3']}}

In [12]:
from __future__ import print_function

In [38]:
import pyfaidx

hg19_fasta = '/projects/ps-yeolab/genomes/hg19/gencode/v19/GRCh37.p13.genome.fa'


genome = pyfaidx.Fasta(hg19_fasta)

In [40]:

# %pdb

In [41]:
import re

In [26]:
se_metadata_subset = se_metadata.head()

exon_cols = [col for col in se_metadata_subset if re.match('^exon\d$', col)]
for exon_col in exon_cols:
    se_metadata_subset[exon_col + '_region'] = se_metadata_subset[exon_col].map(Region)
se_metadata_subset.head(2)

/home/obotvinnik/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,event_length,event_location,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,exons,illegal_junctions,intron_length,intron_location,isoform1_ccdsid,isoform1_gene_id,isoform1_gene_name,isoform1_gene_status,isoform1_gene_type,isoform1_havana_gene,isoform1_havana_transcript,isoform1_level,isoform1_ont,isoform1_tag,isoform1_transcript_id,isoform1_transcript_name,isoform1_transcript_status,isoform1_transcript_type,isoform2_ccdsid,isoform2_gene_id,isoform2_gene_name,isoform2_gene_status,isoform2_gene_type,isoform2_havana_gene,isoform2_havana_transcript,isoform2_level,isoform2_ont,isoform2_tag,isoform2_transcript_id,isoform2_transcript_name,isoform2_transcript_status,isoform2_transcript_type,junction12,junction13,junction23,junctions,strand,exon1_region,exon2_region,exon3_region
event_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
isoform1=junction:chr12:53689424-53690237:+|isoform2=junction:chr12:53689424-53689622:+@exon:chr12:53689623-53689725:+@junction:chr12:53689726-53690237:+,970.0,event:chr12:53689366-53690335:+,exon:chr12:53689366-53689423:+,58.0,exon:chr12:53689623-53689725:+,103.0,exon:chr12:53690238-53690335:+,98.0,exon:chr12:53689366-53689423:+@exon:chr12:5368...,NaN,816.0,intron:chr12:53689423-53690238:+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,junction:chr12:53689424-53689622:+,junction:chr12:53689424-53690237:+,junction:chr12:53689726-53690237:+,junction:chr12:53689424-53689622:+@junction:ch...,+,outrigger.Region (exon:chr12:53689366-53689423:+),outrigger.Region (exon:chr12:53689623-53689725:+),outrigger.Region (exon:chr12:53690238-53690335:+)
isoform1=junction:chr1:32669647-32670198:+|isoform2=junction:chr1:32669647-32669786:+@exon:chr1:32669787-32669980:+@junction:chr1:32669981-32670198:+,1509.0,event:chr1:32669480-32670988:+,exon:chr1:32669480-32669646:+,167.0,exon:chr1:32669787-32669980:+,194.0,exon:chr1:32670199-32670988:+,790.0,exon:chr1:32669480-32669646:+@exon:chr1:326697...,NaN,554.0,intron:chr1:32669646-32670199:+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,junction:chr1:32669647-32669786:+,junction:chr1:32669647-32670198:+,junction:chr1:32669981-32670198:+,junction:chr1:32669647-32669786:+@junction:chr...,+,outrigger.Region (exon:chr1:32669480-32669646:+),outrigger.Region (exon:chr1:32669787-32669980:+),outrigger.Region (exon:chr1:32670199-32670988:+)


In [35]:
from __future__ import print_function

In [43]:
%pdb

Automatic pdb calling has been turned ON


In [65]:
splice_type = 'se'

def maybe_get_sequence(features, fasta, genome, reverse=False):
#     sequences = []
    for feature in features:
        if isinstance(feature, gffutils.Feature):
            sequence = feature.sequence(fasta)
        else:
            sequence = genome[feature.chrom][feature.start:feature.stop].seq
        if reverse:
            sequence = sequence[::-1]
        yield sequence


def maybe_transcribe(features, fasta, genome, reverse=False):
    sequences = maybe_get_sequence(features, reverse=reverse, fasta=fasta, genome=genome)
    sequence = Seq(''.join(sequences), generic_dna)
    if reverse:
        sequence = sequence.complement()
    return sequence
    
    
def maybe_translate(transcribed, first_feature):
    try:
        transcribed = transcribed[int(first_feature.frame):]
    except AttributeError:
        # First feature is probably a UTR and doesn't have a frame
        pass
    translated = transcribed.translate(to_stop=True)
    return str(translated)


def get_feature_in_same_location(db, region, featuretype="CDS", must_be_same=None):
    features = db.region(seqid=region.chrom, start=region.start, end=region.stop, 
                     strand=region.strand, featuretype=featuretype, completely_within=True)
    features = set([f if (f.start == region.start) and (f.end == region.stop) else region for f in features])
    
    if len(features) > 1:
        if must_be_same is not None:
            frames = set([getattr(x, must_be_same) for x in features if hasattr(x, must_be_same)])
        if len(frames) > 1:
            raise ValueError("More than one {featuretype} with different {must_be_same} "
                             "at this exact location ({location})".format(
                    featuretype=featuretype, location=region.name, must_be_same=must_be_same))
    try:
        feature = features.pop()
    except KeyError:
        
        feature = region
    return feature


Start running experiments using the code above

In [66]:
# ---------- Experiments on subset ------------- #

for i, row in se_metadata_subset.iterrows():
    print(i)
    translations = []
    
    event_limit = ':'.join(row['event_location'].split(':')[1:3])
    for isoform, exons in SPLICE_TYPE_ISOFORM_EXONS[splice_type].items():
        print('\t', isoform)
        exon_regions = [row[exon + '_region'] for exon in exons]

        reverse = row['strand'] == '-'
        cds = [get_feature_in_same_location(db, exon_region, must_be_same='frame') for exon_region in exon_regions]
        print('\t\t', cds)
        
        transcribed = maybe_transcribe(cds, hg19_fasta, genome, reverse=reverse)
        print('\t\t\t', transcribed)
        translated = maybe_translate(transcribed, cds[0])
        print('\t\t\t\t', translated)
    

#         exon_features = [maybe_get_feature(db, row[e]) for e in exons]
#         exon_ids = [e.id if isinstance(e, gffutils.Feature) else e for e in exon_features]
# #         print('\t\t', exon_ids)

#         # Have to get parents of all exons because CDSs are the children of transcripts, not exons
#         transcripts = [set(maybe_get_parents(db, exon_feature, featuretype='transcript')) 
#                        for exon_feature in exon_features]
#         transcripts_overlapping = set.intersection(*transcripts)

isoform1=junction:chr12:53689424-53690237:+|isoform2=junction:chr12:53689424-53689622:+@exon:chr12:53689623-53689725:+@junction:chr12:53689726-53690237:+
	 isoform2
		 [<Feature CDS (chr12:53689366-53689423[+]) at 0x2b58994b63d0>, <Feature CDS (chr12:53689623-53689725[+]) at 0x2b58977397d0>, outrigger.Region (exon:chr12:53690238-53690335:+)]
			 TAACATCACGGAGCTGAATCTGCCGCAGCTAGAAATGCTCAAGAACCAGCTGGACCAGGAAGTGGAGTTCTTGTCCACGTCCATTGCTCAGCTCAAAGTGGTACAGACCAAGTATGTGGAAGCCAAGGACTGTCTGAACGTGCTGAACAAGAGCAACGAGGCTGGAGCGCAATGGTGAGATCTTGGCTTACTGCAACCTCCGCCTCCCGAGTTCAAGAGATTCTTCTGCCTCAGTCTCCCGAGTAGCTGGGACTTTAG
				 NITELNLPQLEMLKNQLDQEVEFLSTSIAQLKVVQTKYVEAKDCLNVLNKSNEAGAQW
	 isoform1
		 [<Feature CDS (chr12:53689366-53689423[+]) at 0x2b58950b8410>, outrigger.Region (exon:chr12:53690238-53690335:+)]
			 TAACATCACGGAGCTGAATCTGCCGCAGCTAGAAATGCTCAAGAACCAGCTGGACCAGCTGGAGCGCAATGGTGAGATCTTGGCTTACTGCAACCTCCGCCTCCCGAGTTCAAGAGATTCTTCTGCCTCAGTCTCCCGAGTAGCTGGGACTTTAG
				 NITELNLPQLEMLKNQLDQLERNGEILAYCNLRLPS

AttributeError: 'Region' object has no attribute 'frame'

In [53]:
%pdb

Automatic pdb calling has been turned OFF


In [56]:
get_feature_in_same_location(db, exon_regions[1])

<Feature CDS (chr1:32669787-32669980[+]) at 0x2b58975a4110>

In [51]:
exon_regions

[outrigger.Region (exon:chr1:32669480-32669646:+),
 outrigger.Region (exon:chr1:32669787-32669980:+),
 outrigger.Region (exon:chr1:32670199-32670988:+)]

In [50]:
cds

[<Feature CDS (chr12:53689366-53689423[+]) at 0x2b58994b6450>,
 outrigger.Region (exon:chr12:53690238-53690335:+)]

In [ ]:
## Old code




def maybe_get_feature(db, feature_id):
    try:
        return db[feature_id]
    except gffutils.FeatureNotFoundError:
        return feature_id
    
def maybe_get_children(db, feature_id, **kwargs):
    try:
        return list(db.children(feature_id, **kwargs))
    except gffutils.FeatureNotFoundError:
        return []
    
    
def maybe_get_parents(db, feature_id, **kwargs):
    try:
        return list(db.parents(feature_id, **kwargs))
    except gffutils.FeatureNotFoundError:
        return []
    

    
            
def translate_event(row, splice_type):
#     print(row.name)
    translations = []
    
    event_limit = ':'.join(row['event_location'].split(':')[1:3])
    for isoform, exons in SPLICE_TYPE_ISOFORM_EXONS[splice_type].items():
#         print('\t', isoform)

        reverse = row['strand'] == '-'

        exon_features = [maybe_get_feature(db, row[e]) for e in exons]
        exon_ids = [e.id if isinstance(e, gffutils.Feature) else e for e in exon_features]
#         print('\t\t', exon_ids)

        # Have to get parents of all exons because CDSs are the children of transcripts, not exons
        transcripts = [set(maybe_get_parents(db, exon_feature, featuretype='transcript')) 
                       for exon_feature in exon_features]
        transcripts_overlapping = set.intersection(*transcripts)
#         print('\t\t\toverlapping transcripts:', [t.id for t in transcripts_overlapping])

        transcript_cds_features = [maybe_get_children(db, transcript, featuretype='CDS', 
                                                      order_by='start', reverse=reverse, 
                                                      limit=event_limit)
                        for transcript in transcripts_overlapping]

        for transcript, cds_features in zip(transcripts_overlapping, transcript_cds_features):
            if len(cds_features) == 0:
                continue
            cds_features = [cds if cds.id.replace('CDS', 'exon').startswith(exon_id) else Region(exon_id)
                            for cds, exon_id in zip(cds_features, exon_ids)]
#             print('\t\t\tcds:', cds_features)

            translated = maybe_translate(cds_features, hg19_fasta, genome, reverse)
#             yield [row.name, isoform, translated, transcript.id]
            translations.append([row.name, isoform, translated, transcript.id])
    return translations
#     return pd.DataFrame(translations, columns=['event_id', 'isoform', 'translation', 'transcript'])
            
            


Try with joblib

In [ ]:
import joblib

In [ ]:
%%time
translations = joblib.Parallel(n_jobs=-1)(joblib.delayed(translate_event)(row, 'se') 
                                          for i, row in se_metadata.head(20).iterrows())
# translations_df = pd.DataFrame(list(itertools.chain(*translations)))
# print(translations_df.shape)

Try with yields instead

In [64]:
# se_metadata.loc['isoform1=junction:chr10:101191023-101193302:+|isoform2=junction:chr10:101191023-101191686:+@novel_exon:chr10:101191687-101191705:+@junction:chr10:101191706-101193302:+']

In [65]:
# # %%time 
# translations = se_metadata.head(20).apply(translate_event, axis=1, raw=True)
# translations_df = pd.DataFrame(list(itertools.chain(*translations)))
# print(translations_df.shape)

isoform1=junction:chr10:101191023-101193302:+|isoform2=junction:chr10:101191023-101191686:+@novel_exon:chr10:101191687-101191705:+@junction:chr10:101191706-101193302:+


AttributeError: 'NoneType' object has no attribute 'split'

> <ipython-input-63-d35ab502651b>(62)translate_event()
     60     translations = []
     61 
---> 62     event_limit = ':'.join(row['event_location'].split(':')[1:3])
     63     for isoform, exons in SPLICE_TYPE_ISOFORM_EXONS['se'].items():
     64 #         print('\t', isoform)

ipdb> q


In [72]:
%time translations = se_metadata.head(20).apply(translate_event, axis=1, raw=True)
translations_df = pd.DataFrame(list(itertools.chain(*translations)))
print(translations_df.shape)

CPU times: user 638 ms, sys: 112 ms, total: 750 ms
Wall time: 1.38 s
(46, 4)


1.38s for 20 rows ...


In [73]:
(1.38/20) * se_metadata.shape[0]

379253.67

1099286.0 seconds for all the data

In [74]:
((1.38/20) *se_metadata.shape[0]) /60/60

105.34824166666665

105 hours!!

In [46]:
se_metadata.shape

(5496430, 28)

(46, 4)


,0,1,2,3
0,isoform1=junction:chr10:100189400-100190327:-|...,isoform2,HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIP...,ENST00000338546.5
1,isoform1=junction:chr10:100189400-100190327:-|...,isoform1,HSASSLRPADLLALILLVQDLYPSESTAEDDIQETDSFSLPEEYFT...,ENST00000338546.5
2,isoform1=junction:chr10:100189400-100190327:-|...,isoform2,HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIP...,ENST00000338546.5
3,isoform1=junction:chr10:100189400-100190327:-|...,isoform1,HSASSLRPADLLALILLVQDLYPSESTAEDDIQETDSFSLPEEYFT...,ENST00000338546.5
4,isoform1=junction:chr10:100189400-100190327:-|...,isoform2,HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIP...,ENST00000338546.5


In [45]:
len(translations_df[0].unique())

4

In [42]:
translations_df.loc[translations_df.duplicated()]

,0,1,2,3
2,isoform1=junction:chr10:100189400-100190327:-|...,isoform2,HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIP...,ENST00000338546.5
3,isoform1=junction:chr10:100189400-100190327:-|...,isoform1,HSASSLRPADLLALILLVQDLYPSESTAEDDIQETDSFSLPEEYFT...,ENST00000338546.5
4,isoform1=junction:chr10:100189400-100190327:-|...,isoform2,HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIP...,ENST00000338546.5
5,isoform1=junction:chr10:100189400-100190327:-|...,isoform1,HSASSLRPADLLALILLVQDLYPSESTAEDDIQETDSFSLPEEYFT...,ENST00000338546.5
7,isoform1=junction:chr10:100189400-100190327:-|...,isoform1,HSASSLRPADLLALILLVQDLYPSESTAEDDIQETDSFSLPEEYFT...,ENST00000414009.1
8,isoform1=junction:chr10:100189400-100190327:-|...,isoform1,HSASSLRPADLLALILLVQDLYPSESTAEDDIQETDSFSLPEEYFT...,ENST00000414009.1
14,isoform1=junction:chr10:100189400-100190327:-|...,isoform2,HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIP...,ENST00000361490.4
15,isoform1=junction:chr10:100189400-100190327:-|...,isoform2,HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIP...,ENST00000325103.6
16,isoform1=junction:chr10:100189400-100190327:-|...,isoform1,HSASSLRPADLLALILLVQDLYPSESTAEDDIQETDSFSLPEEYFT...,ENST00000359632.3
17,isoform1=junction:chr10:100189400-100190327:-|...,isoform1,HSASSLRPADLLALILLVQDLYPSESTAEDDIQETDSFSLPEEYFT...,ENST00000361490.4


In [40]:
translations_df.drop_duplicates().shape

(21, 4)

translations['isoform1=junction:chr10:100189400-100190327:-|isoform2=junction:chr10:100189647-100190327:-@exon:chr10:100189548-100189646:-@junction:chr10:100189400-100189547:-'].values

In [100]:
isoform2 = 'HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIPVQQAWSPHSTGPTGGSSAETFGECLFIAINGDHTESEGDLRRKLYVLKYLFEVHFGLVTVDGHLIRK'
isoform1 = 'HSASSLRPADLLALILLVQDLYPSESTAEDDIQFGECLFIAINGDHTESEGDLRRKLYVLKYLFEVHFGLVTVDGHLIRK'

In [84]:
cds_features

[<Feature CDS (chr10:100190328-100190427[-]) at 0x2b5bc077b650>,
 <Feature CDS (chr10:100189548-100189646[-]) at 0x2b5bbf811f90>,
 outrigger.Region (exon:chr10:100188908-100189399:-)]

In [43]:
exon = db['exon:chr10:100190328-100190427:-']
[x.id for x in db.parents(exon)]

['ENST00000361490.4',
 'ENSG00000107521.14',
 'ENST00000325103.6',
 'ENST00000467246.1',
 'ENST00000359632.3',
 'ENST00000478087.1',
 'ENST00000470095.1',
 'ENST00000498219.1',
 'ENST00000338546.5',
 'ENST00000414009.1',
 'ENST00000480020.1']

In [45]:
list(db.children('ENST00000325103.6'))

[<Feature CDS (chr10:100177324-100177483[-]) at 0x2b5bbc6dd5d0>,
 <Feature CDS (chr10:100177932-100178014[-]) at 0x2b5bbf763cd0>,
 <Feature CDS (chr10:100179802-100179915[-]) at 0x2b5bbf176690>,
 <Feature CDS (chr10:100182126-100182270[-]) at 0x2b5bbf762610>,
 <Feature CDS (chr10:100183360-100183425[-]) at 0x2b5bbf7626d0>,
 <Feature CDS (chr10:100183510-100183644[-]) at 0x2b5bbf7627d0>,
 <Feature CDS (chr10:100184070-100184131[-]) at 0x2b5bbf762950>,
 <Feature CDS (chr10:100185298-100185477[-]) at 0x2b5bbf762ad0>,
 <Feature CDS (chr10:100185575-100185742[-]) at 0x2b5bbf762c50>,
 <Feature CDS (chr10:100186972-100187021[-]) at 0x2b5bbf762dd0>,
 <Feature CDS (chr10:100189330-100189399[-]) at 0x2b5bbf762f50>,
 <Feature CDS (chr10:100189548-100189646[-]) at 0x2b5bbf716110>,
 <Feature CDS (chr10:100190328-100190427[-]) at 0x2b5bbf716290>,
 <Feature CDS (chr10:100190888-100191048[-]) at 0x2b5bbf716490>,
 <Feature CDS (chr10:100193740-100193848[-]) at 0x2b5bbf716650>,
 <Feature CDS (chr10:1001

In [36]:
cds_features

[[], [], []]

In [22]:
list(db.region('chr10:100195101-100195171'))

[<Feature gene (chr10:100175955-100206684[-]) at 0x2b5bbc6d7750>,
 <Feature transcript (chr10:100175955-100206667[-]) at 0x2b5bbc6d7b50>,
 <Feature exon (chr10:100195029-100195171[-]) at 0x2b5bbc6dd710>,
 <Feature CDS (chr10:100195029-100195171[-]) at 0x2b5bbc6dd9d0>,
 <Feature transcript (chr10:100175955-100206683[-]) at 0x2b5bbc6ddc50>,
 <Feature transcript (chr10:100176772-100206644[-]) at 0x2b5bbc6dde10>,
 <Feature transcript (chr10:100185663-100206630[-]) at 0x2b5bbc6ddd10>,
 <Feature transcript (chr10:100188908-100206670[-]) at 0x2b5bbc6dd390>,
 <Feature transcript (chr10:100190328-100206619[-]) at 0x2b5bbc6dd810>,
 <Feature transcript (chr10:100194843-100205170[-]) at 0x2b5bbc6d5110>,
 <Feature exon (chr10:100194843-100195171[-]) at 0x2b5bbc6d5610>]

In [ ]:
# %%time

import itertools
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Alphabet import generic_dna, generic_protein


splice_type_isoform_exons = {'SE': {'isoform1': ('exon1', 'exon3'),
                                    'isoform2': ('exon1', 'exon2', 'exon3')},
                             'MXE': {'isoform1': ('exon1', 'exon3', 'exon4'),
                                     'isoform2': ('exon1', 'exon2', 'exon4')}
                            }

# splice_type_isoform1_exons = {'SE': ('exon1', 'exon3'), 
#                               'MXE': ('exon1', 'exon3', 'exon4')}
# splice_type_isoform2_exons = {'SE': ('exon1', 'exon2','exon3'), 
#                               'MXE': ('exon1', 'exon2', 'exon4')}

best_tags = 'appris_principal', 'appris_candidate', 'CCDS', 'exp_conf', 'basic'

def overlap(x, y):
    return not ((x.start > y.stop) or (x.stop < y.start))


def filter_on_tags(features, tags):
    if len(features) == 0:
        return None
    
    if len(features) == 1:
        return features.pop()
    
    feature_tags = map(lambda x: (x, tuple(x.attributes['tag'])) 
                    if 'tag' in x.attributes else (x, ()), features)
#     print 'feature_tags', feature_tags
    for tag in tags:
        for feature, feature_tag in feature_tags:
            if tag in feature_tag:
                return feature
    # If none of the tags are there, return a random feature
#     print features
    return np.random.choice(list(features))

seqrecords = []

for i, (event_id, row) in enumerate(splicing_feature_data.iterrows()):
#     if i > 10:
#         break
    
    if (i+1) % 1000 == 0:
        print i+1
#     exon1 = v19db[row.exon1]
#     exon2 = v19db[row.exon2]
#     exon3 = v19db[row.exon3]
#     print event_id
    
#     exon_trio = exon1, exon2, exon3

    isoform1_exons = splice_type_isoform_exons[row.splice_type]['isoform1']
    isoform2_exons = splice_type_isoform_exons[row.splice_type]['isoform2']
    
    isoform_to_exons = {'isoform1': map(lambda x: v19db[row[x]], isoform1_exons),
                        'isoform2': map(lambda x: v19db[row[x]], isoform2_exons)}
    
    isoform_to_transcripts = dict((k, set.intersection(*map(lambda x: set(v19db.parents(x, featuretype='transcript')), v)))
                                  for k, v in isoform_to_exons.items())
    
#     isoform_to_exons = {'isoform1': (exon1, exon3), 'isoform2': exon_trio}
    
    
    # Make sure isoform1 transcripts are distinct from isoform2
    isoform1_transcripts = isoform_to_transcripts['isoform1'] - isoform_to_transcripts['isoform2']
#     print '\tisoform1_transcripts', ','.join(map(lambda x: x.id, isoform1_transcripts))
#     print '\t', '\t'.join(map(lambda x: str(list(x.attributes['tag'])) if 'tag' in x.attributes else '-', isoform1_transcripts))
    
    # Make sure isoform2 transcripts are distinct from isoform1
    isoform2_transcripts = isoform_to_transcripts['isoform2']
    if row.splice_type == 'MXE':
        isoform2_transcripts = isoform2_transcripts - isoform1_transcripts
#     print '\tisoform2_transcripts', ','.join(map(lambda x: x.id, isoform2_transcripts))
#     print '\t', '\t'.join(map(lambda x: str(list(x.attributes['tag'])) if 'tag' in x.attributes else '-', isoform2_transcripts))
    
    isoform1_transcript = filter_on_tags(isoform1_transcripts, best_tags)
    isoform2_transcript = filter_on_tags(isoform2_transcripts, best_tags)
    
#     if isoform1_transcript is not None:
#         print '\t--isoform1', isoform1_transcript.id,
#         if 'tag' in isoform1_transcript.attributes:
#             print isoform1_transcript.attributes['tag']
#         else:
#             print
#     if isoform2_transcript is not None:
#         print '\t--isoform2', isoform2_transcript.id,
#         if 'tag' in isoform2_transcript.attributes:
#             print isoform2_transcript.attributes['tag']
#         else:
#             print
    
#     isoforms = {'isoform1': isoform1_transcripts, 
#                 'isoform2': isoform2_transcripts}
    isoforms = {'isoform1': isoform1_transcript, 
                'isoform2': isoform2_transcript}
    
    
#     isoform_to_cds = {'isoform1': [], 'isoform2': []}
    for isoform, transcript in isoforms.items():
        if transcript is None:
#             print '\t\t', isoform, 'is None'
            continue
        exons = isoform_to_exons[isoform]
#         print '\t', '\t'.join(map(lambda x: str(list(x.attributes['tag'])) if 'tag' in x.attributes else '-', exons))
        
#         for transcript in transcripts:


        reverse = transcript.strand == '-'
        cdss = v19db.children(transcript, featuretype='CDS', order_by='start', 
                              reverse=reverse)

        cdss = filter(lambda cds: any(map(lambda exon: overlap(cds, exon), exons)), cdss)
#         print cdss
        if len(cdss) == len(exons):
            cds_str = '@'.join(map(lambda x: x.id, cdss))
#                 if cds_str in isoform_to_cds[isoform]:
#                     continue

#                 isoform_to_cds[isoform].append(cds_str)

#                 print '\t\t', '\t'.join(map(lambda x: x.id, exons))
#                 print '\t\t\t', '\t'.join(map(lambda x: x.id, cdss))
#                 if 'tag' in transcript.attributes:
#                     print '\t\t\t', transcript.attributes['tag']
#                 else:
#                     print '\t\t\t-'

            if reverse:
                coding_sequence = Seq(''.join(cds.sequence(hg19_fasta)[::-1] for cds in cdss), generic_dna).complement()
            else:
                coding_sequence = Seq(''.join(cds.sequence(hg19_fasta) for cds in cdss), generic_dna)
#                 print len(coding_sequence)
            coding_sequence = coding_sequence[int(cdss[0].frame):]
            translated = coding_sequence.translate()
            seqrecord = SeqRecord(translated, id='{0}|{1}|{2}'.format(event_id, cds_str, isoform))
#             print '\t\t', translated
            seqrecords.append(seqrecord)
#     pprint(isoform_to_cds)




In [8]:
len(seqrecords)

1193

In [9]:
translated_fasta

'/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations.fa'

In [10]:
with open(translated_fasta, 'w') as f:
    SeqIO.write(seqrecords, f, 'fasta')

In [11]:
! wc -l $translated_fasta

4049 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations.fa


In [12]:
import itertools
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import pandas as pd

translated_df = pd.DataFrame(columns=['isoform1', 'isoform2'])

In [13]:
! wc -l $translated_fasta

4049 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations.fa


In [14]:
with open(translated_fasta) as f:
    for record in SeqIO.parse(f, 'fasta'):
        event_id, cds_str, isoform = record.id.split('|')
        translated_df.loc[event_id, isoform] = str(record.seq)

In [15]:
translated_df.shape

(977, 2)

In [16]:
translated_df.head()

,isoform1,isoform2
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,NaN,HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIP...
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,NaN,NAPEFSIVVLHACAHNPTGIDPTPEQWKQIASVMKHRFLFPFFDSA...
exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+,NaN,ISFSYLCPASWYFTVPTVSPFLRQRVAFLGLFFISCLLLLMLIIDF...
exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,NaN,LPETHQLLQLRRFAHGTAGLVFLTALSGAFVAGLDAGLVYNSFPKM...
exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,FDMVHDPMAALETLLTLGFERVLTSGCDSSALEGLPLIKRLIEQEVV,FDMVHDPMAALETLLTLGFERVLTSGCDSSALEGLPLIKRLIEQAK...


In [6]:
range(3)

[0, 1, 2]

In [7]:
translated_df.dropna().shape

NameError: name 'translated_df' is not defined

In [ ]:
translated_df.columns = [x + '_translation' for x in translated_df]

translated_df.to_csv('{}/protein_translations.csv'.format(alternative_feature_folder))